# Postprocessing of the computed optimal policy

In [1]:
import util as ut
import numpy as np

In [2]:
mdp = ut.get_file("2_mdps.pickle")
print(f"File contains info for {len(mdp)} flights.")

File contains info for 2 flights.


In [3]:
flight_index = 1
costs = mdp[flight_index][1][1]
actions = mdp[flight_index][1][3]
probabilities = mdp[flight_index][1][5]

In [4]:
# obtain parameters
T = len(costs)
print(f"Time horizon: {T}")

max_num_neighbors = 0
for t in range(T):
    list_lengths = [len(lst) for lst in actions[t].values()]
    max_num_neighbors = max(max_num_neighbors, max(list_lengths))
print(f"Max number of neighbors (number of actions needed): {max_num_neighbors}")
numActions = max_num_neighbors

# Find all visible states for all time steps
states = set(key for state_at_time in actions for key in state_at_time.keys())
s2i = {value: index for index, value in enumerate(states)}
i2s = {index: value for index, value in enumerate(states)}
i2s[s2i[13558]] == 13558 # sanity check
numStates = len(states)
print(f"Number of actual states: {numStates}")
print(f"Number of states in infinite horizon MDP (S*T): {numStates*T}")

Time horizon: 24
Max number of neighbors (number of actions needed): 75
Number of actual states: 10861
Number of states in infinite horizon MDP (S*T): 260664


Load policy file

In [5]:
policy_data = np.loadtxt(f"out/{flight_index}_policy.out", dtype=int)

Mappings $\mathcal{T} \times \mathcal{S} \leftrightarrow \{0, \dots, (S\cdot T) - 1\}$

In [6]:
# Col mappings for flattened transition probability tensor (also row mapping for stage cost matrix)
def global_col_P(time, state):
    return time * numStates + state

def inv_global_col_P(index):
    time = index // numStates
    state = index % numStates
    return time, state

#### Extract policy for visible states for each timestep
as a list of dictionaries --> usage `policy[t][s]` where `t` is the timestep and `s` must be an element of `actions[t].keys()`

In [7]:
policy_a = [] # contains action indices for each (t, s)
policy_s = [] # contains state id for each (t, s)

for t in range(T):
    policy_a.append({})
    policy_s.append({})
    visible_states = actions[t].keys()
    for s in visible_states:
        policy_a[t][s] = policy_data[global_col_P(t, s2i[s])]
        assert len(actions[t][s]) > policy_a[t][s] # sanity check that action index is valid
        policy_s[t][s] = actions[t][s][policy_a[t][s]]


In [8]:
policy_a[0]

{12: 18,
 11: 3,
 13: 5,
 15: 4,
 363: 3,
 365: 4,
 367: 3,
 27: 6,
 28: 7,
 29: 5,
 11566: 6,
 11567: 6,
 11568: 5,
 13557: 7,
 13558: 9,
 13559: 6,
 13546: 3,
 13547: 3,
 13548: 4,
 8: 4,
 10: 0,
 26: 3,
 14: 0,
 16: 2,
 30: 2,
 360: 2,
 362: 1,
 364: 2,
 366: 3,
 368: 4,
 9: 2,
 11543: 5,
 11544: 1,
 11545: 1,
 11546: 0,
 17: 3,
 11547: 0,
 8412: 19,
 8414: 19,
 8416: 17,
 10135: 18,
 10136: 19,
 10137: 16,
 8418: 18,
 10138: 15,
 8420: 16,
 10139: 14,
 13545: 2,
 11521: 9,
 11522: 3,
 11523: 2,
 15239: 2,
 15240: 1,
 15241: 1,
 11524: 1,
 15242: 1,
 13549: 5,
 11525: 2,
 15243: 2,
 5669: 32,
 5670: 30,
 5671: 9,
 13556: 3,
 5672: 10,
 5673: 15,
 13560: 6,
 7: 4,
 359: 0,
 361: 1,
 25: 8,
 11564: 7,
 11565: 3,
 13555: 8,
 11542: 8,
 6: 2,
 369: 1,
 11569: 3,
 19: 5,
 371: 5,
 31: 8,
 11570: 8,
 13561: 7,
 11548: 3,
 18: 5,
 6856: 2,
 6857: 3,
 6858: 4,
 6859: 4,
 6860: 4,
 6861: 3,
 6862: 3,
 14985: 2,
 14986: 0,
 14987: 1,
 5987: 0,
 5988: 0,
 5989: 3,
 14988: 2,
 5990: 2,
 14989: 

In [9]:
policy_s[0]

{12: 13548,
 11: 12,
 13: 12,
 15: 12,
 363: 12,
 365: 12,
 367: 12,
 27: 12,
 28: 12,
 29: 12,
 11566: 12,
 11567: 12,
 11568: 12,
 13557: 12,
 13558: 13548,
 13559: 13548,
 13546: 12,
 13547: 12,
 13548: 12,
 8: 11566,
 10: 28,
 26: 10,
 14: 28,
 16: 13559,
 30: 14,
 360: 363,
 362: 365,
 364: 365,
 366: 365,
 368: 367,
 9: 10,
 11543: 27,
 11544: 28,
 11545: 28,
 11546: 28,
 17: 14,
 11547: 29,
 8412: 11566,
 8414: 11567,
 8416: 11567,
 10135: 11566,
 10136: 11567,
 10137: 11567,
 8418: 11567,
 10138: 11567,
 8420: 11568,
 10139: 11568,
 13545: 13557,
 11521: 13557,
 11522: 13558,
 11523: 13558,
 15239: 13557,
 15240: 13558,
 15241: 13558,
 11524: 13558,
 15242: 13558,
 13549: 5672,
 11525: 13559,
 15243: 13559,
 5669: 13546,
 5670: 13547,
 5671: 13735,
 13556: 10,
 5672: 13735,
 5673: 13735,
 13560: 13548,
 7: 8,
 359: 8,
 361: 10,
 25: 8,
 11564: 8,
 11565: 10,
 13555: 8,
 11542: 26,
 6: 11565,
 369: 14,
 11569: 14,
 19: 16,
 371: 16,
 31: 16,
 11570: 16,
 13561: 13549,
 11548: 30

#### Generate flight trajectories

In [10]:
id2name = ut.get_file("num2node_dict.pickle")
name2id = ut.get_file("node2num_dict.pickle")

In [11]:
def generate_flight_trajectory(start_state_name):
    trajectory = [start_state_name]
    for t in range(1,T):
        trajectory.append(id2name[policy_s[t][name2id[trajectory[-1]]]])
    return trajectory

def print_trajectory(trajectory):
    print("{:<7s}{:<10s}{:<10s}".format("Time", "Waypoint", "Flight level"))
    for t in range(T):
        state = trajectory[t]
        name, level = state.split("_")
        print("{:<7d}{:<10s}{:<10s}".format(t+1, name, level))

In [12]:
#traj = generate_flight_trajectory("VEULE_350") # flight 0
traj = generate_flight_trajectory("SITET_300") # flight 1
print(f"Optimal trajectory for flight {flight_index}:\n{32*'-'}")
print_trajectory(traj)

Optimal trajectory for flight 1:
--------------------------------
Time   Waypoint  Flight level
1      SITET     300       
2      INPAX     350       
3      RESMI     350       
4      KOTAP     350       
5      KETEX     350       
6      KUSEK     350       
7      KOTIS     350       
8      KUKOR     350       
9      ADEKA     350       
10     TIS       350       
11     AMDAS     350       
12     VEROT     350       
13     AGREV     350       
14     MTL       350       
15     EVALA     300       
16     XIRBI     250       
17     DOTIG     200       
18     VEDIK     150       
19     AMFOU     150       
20     CUERS     100       
21     AMFOU     50        
22     TIPIK     50        
23     TIPIK     50        
24     TIPIK     50        
